In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [6]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 20 rows

In [7]:
data.groupby('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [8]:
from pyspark.ml.feature import StringIndexer

In [9]:
indexer = StringIndexer(inputCol='Cruise_line',
                       outputCol='Cruise_cat')

In [10]:
indexed = indexer.fit(data).transform(data)

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
indexed.columns

Out[46]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat']

In [13]:
assembler = VectorAssembler(inputCols=[
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Cruise_cat'
], outputCol='features')

In [14]:
output = assembler.transform(indexed)

In [15]:
output.select('features','crew').show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [16]:
final_data = output.select(['features','crew'])

In [17]:
final_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [18]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 111|
 mean|7.594054054054066|
 stddev|3.211194447377766|
 min| 0.59|
 max| 19.1|
+-------+-----------------+

In [20]:
test_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 47|
 mean| 8.2668085106383|
 stddev|4.112136388779681|
 min| 0.6|
 max| 21.0|
+-------+-----------------+

In [21]:
lr = LinearRegression(labelCol='crew')

In [22]:
lr_model = lr.fit(train_data)

In [23]:
test_results = lr_model.evaluate(test_data)

In [24]:
test_results.residuals.show()  #predicted - actual data

+--------------------+
 residuals|
+--------------------+
-0.16348333865789044|
-0.09416246397522698|
 -1.1117516529891383|
 -1.4117814499398609|
 -0.5262985494555963|
 0.22378100962315983|
 -1.3945127692227608|
 -0.8138986796017615|
-0.24397718319833217|
 0.8437217607790526|
 0.38337682395195216|
 -0.7887505243865611|
 0.7451764449938487|
 0.9614707598407986|
0.006986970575223239|
 -0.3432844303843794|
 -1.1709359880091519|
 1.1129299309267786|
-0.32913368616729066|
 0.7728451191631827|
+--------------------+
only showing top 20 rows

In [25]:
test_results.rootMeanSquaredError

Out[62]: 0.6895300828989052

In [26]:
test_results.r2


Out[63]: 0.9712715998231288

In [27]:
 final_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 158|
 mean|7.794177215189873|
 stddev|3.503486564627034|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

In [28]:
unlabeled_data = test_data.select('features')

In [29]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[4.0,220.0,54.0,1...|
[5.0,133.5,39.59,...|
[6.0,30.276999999...|
[6.0,90.0,20.0,9....|
[6.0,112.0,38.0,9...|
[6.0,113.0,37.82,...|
[7.0,89.6,25.5,9....|
[7.0,116.0,31.0,9...|
[7.0,158.0,43.7,1...|
[8.0,91.0,22.44,9...|
[9.0,81.0,21.44,9...|
[9.0,85.0,19.68,9...|
[9.0,88.5,21.24,9...|
[9.0,113.0,26.74,...|
[10.0,77.0,20.16,...|
[10.0,81.76899999...|
[10.0,138.0,31.14...|
[10.0,151.4,26.2,...|
[11.0,90.09,25.01...|
[11.0,91.0,20.32,...|
+--------------------+
only showing top 20 rows

In [30]:
predictions = lr_model.transform(unlabeled_data)

In [31]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[4.0,220.0,54.0,1...| 21.16348333865789|
[5.0,133.5,39.59,...|13.224162463975228|
[6.0,30.276999999...| 4.661751652989138|
[6.0,90.0,20.0,9....| 10.41178144993986|
[6.0,112.0,38.0,9...|11.426298549455597|
[6.0,113.0,37.82,...| 11.77621899037684|
[7.0,89.6,25.5,9....| 11.26451276922276|
[7.0,116.0,31.0,9...|12.813898679601762|
[7.0,158.0,43.7,1...|13.843977183198332|
[8.0,91.0,22.44,9...|10.156278239220947|
[9.0,81.0,21.44,9...| 9.616623176048048|
[9.0,85.0,19.68,9...| 9.47875052438656|
[9.0,88.5,21.24,9...| 9.554823555006152|
[9.0,113.0,26.74,...|11.418529240159202|
[10.0,77.0,20.16,...| 8.993013029424777|
[10.0,81.76899999...| 8.76328443038438|
[10.0,138.0,31.14...|13.020935988009152|
[10.0,151.4,26.2,...| 11.41707006907322|
[11.0,90.09,25.01...| 8.809133686167291|
[11.0,91.0,20.32,...| 9.217154880836818|
+--------------------+------------------+
only showing top 20 rows

In [32]:
from pyspark.sql.functions import corr

In [33]:
data.select(corr('crew','passengers')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [34]:
data.select(corr('crew','cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+